In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# 문장 -> 임베딩 벡터(1차원 숫자 배열)
- openAI API의 키 OPENAI_API_KEY(text-embedding-3-large)를 .env에 추가
- upstage(https://console.upstage.ai)의 키를 .env에 추가 (한글임베딩처리)
# 1. 환경변수 load

In [4]:
from dotenv import load_dotenv
import os
load_dotenv(
    # dotenv_path='.env'
)
openai_key = os.getenv('OPENAI_API_KEY')
upstage_key = os.getenv('UPSTAGE_API_KEY')
print(openai_key[:2])
print(upstage_key[:2])

sk
up


# 2. 유사도 계산하는 방법
- 1. 유클리드 거리 : 두 벡터간 거리가 가까운지
- 2. 코사인 유사도 : 두 벡터간 방향이 유사한지
- 3. dot product : 두 벡터간 곱을 사용하여 거리와 방향을 모두 고려

In [7]:
import numpy as np
def cosine_similarity(vec1, vec2):   # 유사도 계산하는 함수(vec1, vec2-문장을 숫자로 바꾼 값)
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API의 embedding model 사용
- text-embedding-3-large

In [8]:
from openai import OpenAI
openai_client = OpenAI(
    # api_key=openai_key
)

In [9]:
# text-embedding-3-large(과금발생!! 1번만 실행)1000token 이내가 가장 좋다
response = openai_client.embeddings.create(
    input="The king is the prince's father.",
    model="text-embedding-3-large"
)

In [11]:
response.usage  # prompt_tokens=8  입력, total_tokens=8  출력

Usage(prompt_tokens=8, total_tokens=8)

In [13]:
response.data[0].embedding

[0.029435239732265472,
 0.026658575981855392,
 -0.011412999592721462,
 0.020805422216653824,
 0.009366350248456001,
 -0.02719305083155632,
 0.025094259530305862,
 -0.020570773631334305,
 9.603850048733875e-05,
 0.005110103636980057,
 0.03318960219621658,
 0.02014058642089367,
 -0.04854598268866539,
 0.01304901484400034,
 0.034154262393713,
 0.016412297263741493,
 -0.05532469227910042,
 0.030530262738466263,
 -0.040072597563266754,
 -0.009320724755525589,
 -0.012664453126490116,
 -0.040594037622213364,
 -0.0018674039747565985,
 -0.007045946549624205,
 -0.03256387636065483,
 -0.00013697962276637554,
 -0.008805803954601288,
 0.003852131776511669,
 -0.01978861540555954,
 0.0006558726890943944,
 -0.003816282842308283,
 0.012364625930786133,
 0.027062691748142242,
 0.016112469136714935,
 -0.023868877440690994,
 -0.019306283444166183,
 -0.018823953345417976,
 -0.02714090794324875,
 -0.009418494068086147,
 -0.02007540687918663,
 -0.03290281072258949,
 -0.04739881679415703,
 -0.0460691489279270

In [16]:
king_vector = np.array(response.data[0].embedding)  #"The king is ~"문장을 숫자로 바꾼 vector
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.02943524  0.02665858 -0.011413   ...  0.00223567  0.01161506
 -0.00230411]


In [17]:
queen_response = openai_client.embeddings.create(
    input="The queen is the prince's mother",
    model="text-embedding-3-large"
)

In [18]:
queen_response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [20]:
queen_vector = np.array(queen_response.data[0].embedding)
print(queen_vector.shape)
print(queen_vector)

(3072,)
[ 0.01908419  0.00117847 -0.00398764 ...  0.00576628  0.00822409
  0.01082648]


In [21]:
# king_vector와 queen_vector의 유사도 
king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print("king문장과 queen문장의 유사도 :", king_queen_similarity)

king문장과 queen문장의 유사도 : 0.6520014564341383


In [22]:
slave_response = openai_client.embeddings.create(
    input="The slave begs",
    model="text-embedding-3-large"
)

In [24]:
slave_vector = np.array(slave_response.data[0].embedding)
print(slave_vector.shape)
print(slave_vector)

(3072,)
[-0.00868506  0.00686     0.00754387 ... -0.00823628 -0.0112752
  0.00194153]


In [25]:
king_slave_similarity = cosine_similarity(king_vector, slave_vector)
print("king문장과 slave문장의 유사도 :", king_slave_similarity)

king문장과 slave문장의 유사도 : 0.14976215541942633


In [ ]:
# 한국어 문장을 벡터로 바꿔도 유사도가 비슷

In [26]:
response = openai_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="text-embedding-3-large"
)

In [29]:
kor_king_vector = np.array(response.data[0].embedding)
print(kor_king_vector.shape)
print(kor_king_vector)

(3072,)
[0.01460524 0.0081415  0.00299669 ... 0.00721534 0.01149217 0.00849063]


In [30]:
response = openai_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="text-embedding-3-large"
)

In [31]:
kor_queen_vector = np.array(response.data[0].embedding)
print(kor_queen_vector.shape)
print(kor_queen_vector)

(3072,)
[-0.0064186   0.0129057   0.00533253 ... -0.00637457 -0.00048678
  0.03369767]


In [34]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(kor_king_vector, kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.5745403094020906


In [35]:
response = openai_client.embeddings.create(
    input="노예가 구걸한다",
    model="text-embedding-3-large"
)

In [36]:
kor_slave_vector = np.array(response.data[0].embedding)
print(kor_slave_vector.shape)
print(kor_slave_vector)

(3072,)
[ 0.00525312  0.0029851   0.00266439 ... -0.01132859 -0.00260683
  0.01133517]


In [37]:
print("왕문장과 노예문장의 유사도 :", cosine_similarity(kor_king_vector, kor_slave_vector))

왕문장과 노예문장의 유사도 : 0.13559169583133263


In [38]:
king_vector

array([ 0.02943524,  0.02665858, -0.011413  , ...,  0.00223567,
        0.01161506, -0.00230411], shape=(3072,))

In [39]:
kor_king_vector

array([0.01460524, 0.0081415 , 0.00299669, ..., 0.00721534, 0.01149217,
       0.00849063], shape=(3072,))

In [42]:
# king_vector과 kor_king_vector의 유사도
print("king문장과 왕문장의 유사도 :", cosine_similarity(king_vector, kor_king_vector))

king문장과 왕문장의 유사도 : 0.6447228905822825


# 4. upstage의 embedding model 사용
- 한국어에서는 openai 보다 성능이 좋다

In [43]:
upstage_client = OpenAI(
    api_key = upstage_key,
    base_url="https://api.upstage.ai/v1"
)

In [44]:
response = upstage_client.embeddings.create(
    input="The king is prince's father",
    model="embedding-query"
)

In [47]:
up_king_vector = np.array(response.data[0].embedding)
print(up_king_vector.shape)
print(up_king_vector)

(4096,)
[ 8.99434090e-05 -3.04260254e-02 -2.86674500e-03 ... -7.19451904e-03
  1.04446411e-02  3.69873047e-02]


In [49]:
response = upstage_client.embeddings.create(
    input="The queen is prince's mother",
    model="embedding-query"
)

In [50]:
up_queen_vector = np.array(response.data[0].embedding)
print(up_queen_vector.shape)
print(up_queen_vector)

(4096,)
[-0.00610672 -0.01780602  0.00733072 ...  0.00306    -0.00264868
  0.04816383]


In [51]:
print("king문장과 queen문장의 유사도 :", cosine_similarity(up_king_vector, up_queen_vector))

king문장과 queen문장의 유사도 : 0.6696365016255479


In [52]:
response = upstage_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="embedding-query"
)

In [53]:
up_kor_king_vector = np.array(response.data[0].embedding)
print(up_kor_king_vector.shape)

(4096,)


In [54]:
response = upstage_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="embedding-query"
)

In [55]:
up_kor_queen_vector = np.array(response.data[0].embedding)
print(up_kor_queen_vector.shape)

(4096,)


In [56]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(up_kor_king_vector,
                                            up_kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.6502462939208556


In [57]:
print("king문장과 왕문장의 유사도 :", cosine_similarity(up_king_vector, 
                                              up_kor_king_vector))

king문장과 왕문장의 유사도 : 0.8494802298580414


In [58]:
print("queen문장과 여왕문장의 유사도 :", cosine_similarity(up_queen_vector, 
                                                up_kor_queen_vector))

queen문장과 여왕문장의 유사도 : 0.8024567619934905
